## Libraries 📚

In [1]:
import re
import fitz
import spacy 
from spacy.matcher import Matcher
from  transformers import LayoutLMForTokenClassification, LayoutLMTokenizer
import torch

c:\Users\Jorge\anaconda3\envs\Extract\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False


## ---------use🤖----------

In [3]:
# Cargar modelo de spaCy
nlp = spacy.load("es_core_news_sm")

# Cargar modelo de LayoutLM
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased")

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#  -------- Functions


## Fuction with regex 📌


In [ ]:
def extract_with_regex(pattern, text):
    """Extrae información usando una expresión regular."""
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1) if match else "No encontrado"

## Function to extract text of pdf 📌

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

## Funtion clean text 📌

In [ ]:
def limpiar_texto(texto):
    """Normaliza y limpia el texto para mejorar la extracción de metadatos."""
    texto = re.sub(r'\s+', ' ', texto)  # Remueve espacios extra
    texto = re.sub(r'[^\x20-\x7E\xC0-\xFF]', '', texto)  # Remueve caracteres especiales
    return texto.strip()

## Function read a pdf 📌

In [ ]:
def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto limpio."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text("text") + "\n"
    return limpiar_texto(texto)

## Function from number to whole number 📌

In [ ]:
def whole_number(romano):
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
            'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

## Delete spaces in titles 📌

In [ ]:
def normalizar_espacios_titulo(texto):
    """
    Corrige textos OCR donde hay espacios innecesarios entre letras en frases clave.
    """
    frases_clave = [
        "SENADO DE LA REPÚBLICA",
        "CÁMARA DE REPRESENTANTES",
        "PROYECTOS DE LEY"
    ]

    for frase in frases_clave:
        # change the frase
        frase_regex = r'\s*'.join(frase)  # change in "S\s*E\s*N\s*A\s*D\s*O\s* ..."
        texto = re.sub(frase_regex, frase, texto, flags=re.IGNORECASE)  #remplaze

    return texto

# -------

## Principall function to extract metadata of PDF 🐦‍⬛

In [ ]:
def extract_metadata_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    metadata = extract_metadata(text)

    # Convertir a JSON
    metadata_json = json.dumps(metadata, indent=4, ensure_ascii=False)
    return metadata_json

## ---------use🤖----------

In [ ]:
pdf_path = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf"
metadata_json = extract_metadata_from_pdf(pdf_path)
print(metadata_json)